* compares mean of a cat to mean of dependent var over all previous cats seen for  coef
* forward helmert coding is vice versa
* this contrast coding variant tries to capture relationshios btw cat or diff btw ordinal cat

In [1]:
#7th constrast coding techcnique

import pandas as pd
import numpy as np
from statsmodels.formula.api import ols

In [2]:
car_data = pd.read_csv("./datasets/auto-mpg.csv", na_values="?")

car_data.sample(8)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
39,14.0,8,400.0,175.0,4464,11.5,71,1,pontiac catalina brougham
154,15.0,6,250.0,72.0,3432,21.0,75,1,mercury monarch
311,32.1,4,98.0,70.0,2120,15.5,80,1,chevrolet chevette
337,32.4,4,107.0,72.0,2290,17.0,80,3,honda accord
53,31.0,4,71.0,65.0,1773,19.0,71,3,toyota corolla 1200
345,35.1,4,81.0,60.0,1760,16.1,81,3,honda civic 1300
101,23.0,6,198.0,95.0,2904,16.0,73,1,plymouth duster
54,35.0,4,72.0,69.0,1613,18.0,71,3,datsun 1200


In [3]:
# get just two cols to use as X and y, cylinder col is the col that will be helmert encoded
car_data = car_data[["mpg", "cylinders"]]

In [4]:
car_data.dropna(inplace = True)

In [5]:
car_data.sample(6)

,mpg,cylinders
300,23.9,8
171,24.0,4
118,24.0,4
47,19.0,6
361,25.4,6
100,18.0,6


In [7]:
car_data.cylinders.unique()  # since unique num ain't continous, we can say it a cat data

array([8, 4, 6, 3, 5], dtype=int64)

In [8]:
# sort cars based on numner of cylinders
car_data.sort_values(by = ["cylinders"], inplace = True)
car_data.reset_index(inplace=True, drop=True)
car_data.head(10)

,mpg,cylinders
0,18.0,3
1,19.0,3
2,23.7,3
3,21.5,3
4,27.5,4
5,30.0,4
6,25.1,4
7,36.1,4
8,39.4,4
9,36.1,4


In [9]:
car_data.mean()

mpg          23.514573
cylinders     5.454774
dtype: float64

In [13]:
# calc avg mpg for each cyclinder cat
car_data_grouped = car_data.groupby(by = ["cylinders"]).mean()

car_data_grouped

,mpg
cylinders,
3,20.550000
4,29.286765
5,27.366667
6,19.985714
8,14.963107


In [14]:
# get means of all cat(i.e mean of grouped means(mean mpg))
car_data_grouped["mpg"].mean()

22.430450490875963

In [15]:
# calc coef of cat 4 using helmer coding manually, divide by 2 since working with 2 cats
coefficient_cylinder_4 = (car_data_grouped.loc[4]["mpg"] - car_data_grouped.loc[3]["mpg"]) / 2

coefficient_cylinder_4

4.368382352941179

In [16]:
mean_34 = (car_data_grouped.loc[3]["mpg"] + car_data_grouped.loc[4]["mpg"]) / 2 

coefficient_cylinder_5 = (car_data_grouped.loc[5]["mpg"] - mean_34) / 3 # 3 since 3 cat involved
coefficient_cylinder_5

0.8160947712418279

**helmert encoding using statsmodels**

In [17]:
mod = ols("mpg ~ C(cylinders, Helmert)", data = car_data)

res = mod.fit()
res.summary()

# intercept is same as cat means

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.637
Model:                            OLS   Adj. R-squared:                  0.634
Method:                 Least Squares   F-statistic:                     172.6
Date:                Tue, 12 Jan 2021   Prob (F-statistic):           3.68e-85
Time:                        15:25:50   Log-Likelihood:                -1180.8
No. Observations:                 398   AIC:                             2372.
Df Residuals:                     393   BIC:                             2392.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                     22.4305      0.739     30.353      0.000      20.978      23.883
C(cylinders, Helmert)[H.4]     4.3684      1.194      3.657      0.000       2.020       6.717
C(cylinders, Helmert)[H.5]     0.8161      0.994      0.821      0.412      -1.138       2.770
C(cylinders, Helmert)[H.6]    -1.4372      0.329     -4.371      0.000      -2.084      -0.791
C(cylinders, Helmert)[H.8]    -1.8668      0.206     -9.079      0.000      -2.271      -1.463
==============================================================================
Omnibus:                       48.011   Durbin-Watson:                   1.255
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.510
Skew:                           0.793   Prob(JB):                     2.96e-16
Kurtosis:                       4.341   Cond. No.                         12.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

***
**helmer encoding using cat-encoder**

In [18]:
import category_encoders as ce

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [19]:
ce_helmert = ce.HelmertEncoder(cols = ["cylinders"])
car_he = ce_helmert.fit_transform(car_data)

car_he.sample(6)

D:\Anaconda_3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,intercept,mpg,cylinders_0,cylinders_1,cylinders_2,cylinders_3
131,1,40.9,1.0,-1.0,-1.0,-1.0
63,1,30.0,1.0,-1.0,-1.0,-1.0
199,1,31.0,1.0,-1.0,-1.0,-1.0
145,1,29.8,1.0,-1.0,-1.0,-1.0
151,1,25.0,1.0,-1.0,-1.0,-1.0
203,1,31.0,1.0,-1.0,-1.0,-1.0


In [20]:
X = car_he.drop(columns = ["mpg"], axis = 1)
y = car_he.mpg

In [21]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression(fit_intercept = False)
linear_model.fit(X, y)

print("Training_score: ", linear_model.score(X, y))

Training_score:  0.6372420899156167


In [22]:
linear_model.coef_

array([22.43045049,  4.36838235,  0.81609477, -1.43719071, -1.86683592])